In [2]:
using LinearAlgebra
using CUDA           # GPU computing
using GLMakie        # high‑performance 3‑D plotting
using ColorSchemes
using GeometryBasics
import ProgressMeter
using Markdown

In [3]:
const Nx, Ny, Nz = 64, 64, 64          # grid size
threads = (8, 8, 8)                                # 512 threads per block
blocks  = (cld(Nx,threads[1]),
           cld(Ny,threads[2]),
           cld(Nz,threads[3]))                     # as many blocks as needed
############ Physical & numerical constants #############
const dx  = 0.1f0                      # spacing
const dt  = 0.001f0                    # time step
const nsteps      = 2000                # total steps
const frame_every = 5                  # dump every n steps

5

In [ ]:
const m   = 0.5f0
############ Allocate GPU arrays ########################
ϕ   = CUDA.zeros(Float32, Nx, Ny, Nz)
∂ₜϕ  = CUDA.zeros(Float32, Nx, Ny, Nz)  # time derivative

############ Initial condition (Gaussian pulse) #########
function init_gaussian!(field, σ, A, x0=0f0, y0=0f0, z0=0f0)
    i = (blockIdx().x-1)*blockDim().x + threadIdx().x
    j = (blockIdx().y-1)*blockDim().y + threadIdx().y
    k = (blockIdx().z-1)*blockDim().z + threadIdx().z
    nx, ny, nz = size(field)
    if 1 ≤ i ≤ nx && 1 ≤ j ≤ ny && 1 ≤ k ≤ nz
        r2 = (i - nx/2 - x0)^2 + (j - ny/2 - y0)^2 + (k - nz/2 - z0)^2
        field[i,j,k] += A * exp(-r2 / (2σ^2))
    end
    return
end
σ = 3f0
amp = 1f0
@cuda threads=threads blocks=blocks init_gaussian!(ϕ, σ, -amp, 0, 0, -σ/2)
@cuda threads=threads blocks=blocks init_gaussian!(ϕ, σ, amp, 0, 0, σ/2)
# @cuda threads=threads blocks=blocks init_gaussian!(∂ₜϕ, σ, -A, 0, 0, σ*3/2)
@cuda threads=threads blocks=blocks init_gaussian!(∂ₜϕ, σ, amp, 0, 0, 0)

synchronize()
############ Makie set‑up ################################
fig = Figure(size = (700, 500))
ax  = Axis3(fig[1,1]; 
    perspectiveness=0.8, 
    title="ϕ field evolution",
    limits=(1, Nx, 1, Ny, 1, Nz)  # Set explicit limits
)


# Convert 3D array to vector of points
function array_to_points(arr)
    points = Point3f[]
    for i in 1:size(arr,1), j in 1:size(arr,2), k in 1:size(arr,3)
        push!(points, Point3f(i, j, k))
    end
    return points
end

# Initialize points and colors
points = array_to_points(Array(ϕ))
# Create explicit black colors with zero alpha
init_colors = [RGBAf(0.0f0, 0.0f0, 0.0f0, 0.0f0) for _ in 1:length(points)]
markers = scatter!(ax, points; 
    markersize = 4,
    color = init_colors,
    transparency = true
)

Scatter{Tuple{Vector{Point{3, Float32}}}}

In [ ]:
############ Finite‑difference kernel ###################
function update!(∂ₜϕ, ϕ, dx2, m2)
    i = (blockIdx().x-1)*blockDim().x + threadIdx().x
    j = (blockIdx().y-1)*blockDim().y + threadIdx().y
    k = (blockIdx().z-1)*blockDim().z + threadIdx().z
    nx, ny, nz = size(ϕ)
    if 2 ≤ i ≤ nx-1 && 2 ≤ j ≤ ny-1 && 2 ≤ k ≤ nz-1
        lap = (ϕ[i-1,j,k] - 2f0*ϕ[i,j,k] + ϕ[i+1,j,k] +
               ϕ[i,j-1,k] - 2f0*ϕ[i,j,k] + ϕ[i,j+1,k] +
               ϕ[i,j,k-1] - 2f0*ϕ[i,j,k] + ϕ[i,j,k+1]) / dx2
        ∂ₜϕ[i,j,k] += (lap - m2*ϕ[i,j,k]) * dt      # ∂ₜϕ update
    end
    return
end



# Helper that converts GPU array → positions + colors
function snapshot!(markers, ϕh)
    # Create colors with varying transparency - explicitly use zeros for RGB (black)
    cols = [RGBAf(ϕh > 0 ? 0.0f0 : 1.0f0, 0.0f0, ϕh < 0 ? 0.0f0 : 1.0f0, abs(ϕh)) for ϕh in vec(ϕh)]
    # Try direct attribute access
    markers.color = cols
    return
end

snapshot! (generic function with 1 method)

In [ ]:
############ Time integration + recording ################
iter_num = ceil(Int,nsteps/frame_every)
p = ProgressMeter.Progress(iter_num)
GLMakie.record(fig, "phi3d.mp4", 1:iter_num) do frame
    for _ in 1:frame_every
        # (a) compute ∂ₜϕ
        @cuda threads=threads blocks=blocks update!(∂ₜϕ, ϕ, dx^2, m^2)
        # (b) integrate ϕ ← ϕ + ∂ₜϕ*dt
        @. ϕ  = ϕ + ∂ₜϕ*dt        # broadcast runs on GPU automatically :contentReference[oaicite:3]{index=3}
    end
    # copy to CPU & refresh plot every few steps
    ϕh = Array(ϕ)
    snapshot!(markers, ϕh)
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

md"<video src=phi3d.mp4 controls></video>"

Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


<video src=phi3d.mp4 controls></video>


# Electrodynamics!!

I will create Dirac spinor field with active local U(1) symmetry

In [10]:
# Pauli matrices

σx = [0 1;
      1 0]

σy = [0 -im;
       im 0]

σz = [1 0;
      0 -1]

σ0 = [1 0;
      0 1]

# Gamma matrices
# (first make empty 4x4 matrices and set upper-right and lower-left 2x2 blocks to σx, σy, σz)
γ0 = zeros(ComplexF32, 4, 4)
γ0[3:4, 1:2] .= σ0
γ0[1:2, 3:4] .= σ0
γ1 = zeros(ComplexF32, 4, 4)
γ1[3:4, 1:2] .= -σx
γ1[1:2, 3:4] .= σx
γ2 = zeros(ComplexF32, 4, 4)
γ2[3:4, 1:2] .= -σy
γ2[1:2, 3:4] .= σy
γ3 = zeros(ComplexF32, 4, 4)
γ3[3:4, 1:2] .= -σz
γ3[1:2, 3:4] .= σz
γ5 = zeros(ComplexF32, 4, 4)
γ5[1:2, 1:2] .= -σ0
γ5[3:4, 3:4] .= σ0


γ2

4×4 Matrix{ComplexF32}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0-1.0im
 0.0+0.0im  0.0+0.0im  0.0+1.0im  0.0+0.0im
 0.0+0.0im  0.0+1.0im  0.0+0.0im  0.0+0.0im
 0.0-1.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [ ]:
ψ = CUDA.zeros(ComplexF32, Nx, Ny, Nz, 4) # 4-component Dirac spinor
A = CUDA.zeros(ComplexF32, Nx, Ny, Nz, 4) # photon field A^μ
∂ₜA = CUDA.zeros(ComplexF32, Nx, Ny, Nz, 4) # time derivative of A^μ

# electron mass in GeV
const mₑ = 0.000511f0
const qₑ = 0.3029f0 # electron charge

0.3029f0

In [16]:
function init_field!(ψ, A, ∂ₜA)
    i = (blockIdx().x-1)*blockDim().x + threadIdx().x
    j = (blockIdx().y-1)*blockDim().y + threadIdx().y
    k = (blockIdx().z-1)*blockDim().z + threadIdx().z
    # nx, ny, nz = size(A[:,:,:,1])
    if 1 ≤ i ≤ Nx && 1 ≤ j ≤ Ny && 1 ≤ k ≤ Nz
        # TODO: initialize ψ, A, ∂ₜA
    end
    return
end

@cuda threads=threads blocks=blocks init_field!(ψ, A, ∂ₜA)
synchronize()

fig = Figure(size = (700, 500))
ax  = Axis3(fig[1,1]; 
    perspectiveness=0.8, 
    title="ψ field evolution",
    limits=(1, Nx, 1, Ny, 1, Nz)  # Set explicit limits
)


# Convert 3D array to vector of points
function array_to_points(arr)
    points = Point3f[]
    for i in 1:size(arr,1), j in 1:size(arr,2), k in 1:size(arr,3)
        push!(points, Point3f(i, j, k))
    end
    return points
end


# Initialize points and colors
points = array_to_points(Array(ψ[:,:,:,1]))
# Create explicit black colors with zero alpha
init_colors = [RGBAf(0.0f0, 0.0f0, 0.0f0, 0.0f0) for _ in 1:length(points)]
markers = scatter!(ax, points; 
    markersize = 4,
    color = init_colors,
    transparency = true
)

Scatter{Tuple{Vector{Point{3, Float32}}}}

In [ ]:
############ Finite‑difference kernel ###################
function update!(ψ, A, ∂ₜA, dx)
    i = (blockIdx().x-1)*blockDim().x + threadIdx().x
    j = (blockIdx().y-1)*blockDim().y + threadIdx().y
    k = (blockIdx().z-1)*blockDim().z + threadIdx().z
    if 2 ≤ i ≤ nx-1 && 2 ≤ j ≤ ny-1 && 2 ≤ k ≤ nz-1
        # ψ at i,j,k
        psi = ψ[i,j,k,:]
        ∂₁ψ = (ψ[i+1,j,k,:] - psi) / dx
        ∂₂ψ = (ψ[i,j+1,k,:] - psi) / dx
        ∂₃ψ = (ψ[i,j,k+1,:] - psi) / dx
        # psi at i,j,k
        ψ[i,j,k,:] += (
            # -imγᵗψ
            -im * mₑ * γ0 * psi
            # -iqγᵗγᵘψAᵤ
            -im * qₑ * γ0 * γ0 * psi * A[i,j,k,0]
            +im * qₑ * γ0 * γ1 * psi * A[i,j,k,1]
            +im * qₑ * γ0 * γ2 * psi * A[i,j,k,2]
            +im * qₑ * γ0 * γ3 * psi * A[i,j,k,3]
            # -γᵗγᵃ∂ₐψ
            -γ0 * γ1 * ∂₁ψ
            -γ0 * γ2 * ∂₂ψ
            -γ0 * γ3 * ∂₃ψ
        ) * dt
        # A at i,j,k
        A_point = A[i,j,k,:]
        # four-vector of each component's laplacian
        lapA = (A[i-1,j,k,:] - 2f0*A_point + A[i+1,j,k,:] +
               A[i,j-1,k,:] - 2f0*A_point + A[i,j+1,k,:] +
               A[i,j,k-1,:] - 2f0*Apoint + A[i,j,k+1,:]) ./ dx^2
        # caution: index of A start from 1, but 1 is time component (corresponding to 0 in 4-vector)
        ∂ₜA[i,j,k,1] += (
            # qψ†γᵗγᵘψ
            qₑ * ψ[i,j,k,:] * γ0 * γ0 * ψ[i,j,k,:]
            # lap Aᵘ
            + lapA[1]
            # 
        ) * dt
        
    end
    return
end



# Helper that converts GPU array → positions + colors
function snapshot!(markers, ϕh)
    # Create colors with varying transparency - explicitly use zeros for RGB (black)
    cols = [RGBAf(ϕh > 0 ? 0.0f0 : 1.0f0, 0.0f0, ϕh < 0 ? 0.0f0 : 1.0f0, abs(ϕh)) for ϕh in vec(ϕh)]
    # Try direct attribute access
    markers.color = cols
    return
end

In [ ]:
############ Time integration + recording ################
iter_num = ceil(Int,nsteps/frame_every)
p = ProgressMeter.Progress(iter_num)
GLMakie.record(fig, "phi3d.mp4", 1:iter_num) do frame
    for _ in 1:frame_every
        # (a) compute ∂ₜϕ
        @cuda threads=threads blocks=blocks update!(ψ, A, ∂ₜA, dx)
        # (b) integrate ϕ ← ϕ + ∂ₜϕ*dt
        @. A = A + ∂ₜA * dt       # broadcast runs on GPU automatically :contentReference[oaicite:3]{index=3}
    end
    # copy to CPU & refresh plot every few steps
    ϕh = Array(ϕ)
    snapshot!(markers, ϕh)
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

md"<video src=phi3d.mp4 controls></video>"